In [1]:
import time
import sklearn
import numpy as np
import pandas as pd

import sys
sys.path.append("..")
from baggingPU import BaggingClassifierPU

from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import random

In [2]:
def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    columnwidth = max([len(x) for x in labels]) + 4
    empty_cell = " " * columnwidth
    print("    " + empty_cell, end=' ')
    for label in labels:
        print("%{0}s".format(columnwidth) % 'pred_' + label, end=" ")
    print()

    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % 'true_' + label1, end=" ")
        for j in range(len(labels)):
            cell = "%{0}.1f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            if cell:
                print(cell, end=" ")
        print()

# import data

In [3]:
#df_raw = pd.read_csv('../data/w-dependence.csv')

df_raw = pd.read_csv('../data/1place-independence.csv')

#df_raw = pd.read_csv('../data/w-related.csv')


df_raw['label'] = df_raw['label'].astype("int")
print(df_raw.label.value_counts())
print('Has null values', df_raw.isnull().values.any())

1    10000
0    10000
Name: label, dtype: int64
Has null values False


In [4]:
df_raw.head(10)

,p1,p2,p3,p4,label
0,1,0,0,0,1
1,0,1,1,0,1
2,1,0,1,0,1
3,0,0,0,1,1
4,0,1,2,0,1
5,1,0,2,0,1
6,0,0,1,1,1
7,0,1,3,0,1
8,1,0,3,0,1
9,0,0,2,1,1


In [5]:
print(df_raw.iloc[:10000,:])
df_max = np.amax(df_raw.iloc[:10000,[2]])
print(df_max)

      p1  p2    p3  p4  label
0      1   0     0   0      1
1      0   1     1   0      1
2      1   0     1   0      1
3      0   0     0   1      1
4      0   1     2   0      1
...   ..  ..   ...  ..    ...
9995   1   0  3332   0      1
9996   0   0  3331   1      1
9997   0   1  3333   0      1
9998   1   0  3333   0      1
9999   0   0  3332   1      1

[10000 rows x 5 columns]
p3    3333
dtype: int64


In [6]:
print(df_raw.iloc[10000:,:])
print(np.amax(df_raw.iloc[10000:,[2]]))

         p1    p2    p3    p4  label
10000     0     0     0     0      0
10001     0     0     1     0      0
10002     0     1     0     0      0
10003     0     1     0     1      0
10004     0     1     1     1      0
...     ...   ...   ...   ...    ...
19995  1948  3267  3248    22      0
19996   678  2207  1707  1330      0
19997   796  3271  1813  1477      0
19998  2163   614  2392  2879      0
19999  1420  2443  2561  1150      0

[10000 rows x 5 columns]
p3    3333
dtype: int64


In [7]:
neg_process = df_raw.iloc[10000:,:].copy()
print(neg_process)

for i in range(10000):
    if neg_process.iloc[i,2] > df_max[0]:
        neg_process.iloc[i,2]  = int(neg_process.iloc[i,2] / df_max[0])

         p1    p2    p3    p4  label
10000     0     0     0     0      0
10001     0     0     1     0      0
10002     0     1     0     0      0
10003     0     1     0     1      0
10004     0     1     1     1      0
...     ...   ...   ...   ...    ...
19995  1948  3267  3248    22      0
19996   678  2207  1707  1330      0
19997   796  3271  1813  1477      0
19998  2163   614  2392  2879      0
19999  1420  2443  2561  1150      0

[10000 rows x 5 columns]


In [8]:
neg_process.drop_duplicates(inplace=True)
print(neg_process)
print(len(neg_process))
print(np.amax(neg_process))

         p1    p2    p3    p4  label
10000     0     0     0     0      0
10001     0     0     1     0      0
10002     0     1     0     0      0
10003     0     1     0     1      0
10004     0     1     1     1      0
...     ...   ...   ...   ...    ...
19995  1948  3267  3248    22      0
19996   678  2207  1707  1330      0
19997   796  3271  1813  1477      0
19998  2163   614  2392  2879      0
19999  1420  2443  2561  1150      0

[10000 rows x 5 columns]
10000
p1       3333
p2       3332
p3       3333
p4       3333
label       0
dtype: int64


In [9]:
df_process = pd.concat([df_raw.iloc[:10000,:], neg_process])
print(df_process)

         p1    p2    p3    p4  label
0         1     0     0     0      1
1         0     1     1     0      1
2         1     0     1     0      1
3         0     0     0     1      1
4         0     1     2     0      1
...     ...   ...   ...   ...    ...
19995  1948  3267  3248    22      0
19996   678  2207  1707  1330      0
19997   796  3271  1813  1477      0
19998  2163   614  2392  2879      0
19999  1420  2443  2561  1150      0

[20000 rows x 5 columns]


In [10]:
x_data = df_process.iloc[:,:-1]
y_data = df_process.iloc[:,-1]

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=7)

In [11]:
x_train_input = pd.concat([x_train.iloc[:,:], y_train],axis=1)
x_test_input = pd.concat([x_test.iloc[:,:],y_test],axis=1)

print(len(x_train_input))
print(len(x_test_input))

print(x_train_input.label.value_counts())
print('Has null values', x_train_input.isnull().values.any())

print(x_test_input.label.value_counts())
print('Has null values', x_test_input.isnull().values.any())

16000
4000
1    8077
0    7923
Name: label, dtype: int64
Has null values False
0    2077
1    1923
Name: label, dtype: int64
Has null values False


In [12]:
x_train_input.head(10)

,p1,p2,p3,p4,label
8118,0,0,2705,1,1
10171,0,0,2141,0,0
9399,0,0,3132,1,1
12669,1,1,2336,0,0
18809,1457,3115,942,1111,0
13869,0,0,1609,0,0
3320,1,0,1107,0,1
14689,1,1,1319,0,0
13087,1,1,2692,1,0
15992,515,833,2507,1884,0


In [13]:
x_test_input.head(10)

,p1,p2,p3,p4,label
14356,1,1,3189,1,0
3439,0,1,1147,0,1
12153,1,0,2606,1,0
15029,2966,1336,754,1574,0
18549,3088,1378,1096,3034,0
15762,622,795,773,1953,0
12313,0,1,1788,1,0
16034,704,1577,717,2627,0
11496,1,0,1418,1,0
5653,0,1,1885,0,1


In [14]:
df = x_train_input.copy()

NON_LBL = [c for c in df.columns if c != 'label']
X = df[NON_LBL]
y = df['label']

y_orig = y.copy()

hidden_size = 5000
y.loc[
    np.random.choice(
        y[y == 1].index, 
        replace = False, 
        size = hidden_size
    )
] = 0

print(X)
print(y)

         p1    p2    p3    p4
8118      0     0  2705     1
10171     0     0  2141     0
9399      0     0  3132     1
12669     1     1  2336     0
18809  1457  3115   942  1111
...     ...   ...   ...   ...
13927     1     1  1066     0
919       0     1   307     0
5699      1     0  1900     0
10742     0     1  2930     1
16921  2113   604  1171  1381

[16000 rows x 4 columns]
8118     1
10171    0
9399     1
12669    0
18809    0
        ..
13927    0
919      0
5699     1
10742    0
16921    0
Name: label, Length: 16000, dtype: int64


In [15]:
pd.Series(y).value_counts()

0    12923
1     3077
Name: label, dtype: int64

In [16]:
print('- %d samples and %d features' % (X.shape))
print('- %d positive out of %d total before hiding labels' % (sum(df_process.label), len(df_process.label)))
print('- %d positive out of %d total after hiding labels' % (sum(y), len(y)))

- 16000 samples and 4 features
- 10000 positive out of 20000 total before hiding labels
- 3077 positive out of 16000 total after hiding labels


# Trainning directly

In [17]:
print('Training XGboost model ...')

import xgboost as xgb

model = xgb.XGBClassifier()

#from sklearn.linear_model import LogisticRegression
#model = LogisticRegression()


#from sklearn.neural_network import MLPClassifier
#model = MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(1), random_state=1,max_iter=100000)

model.fit(X, y)

print('Done')

Training XGboost model ...
Done


In [18]:
print('---- {} ----'.format('XGboost model'))
print(print_cm(sklearn.metrics.confusion_matrix(y_test, model.predict(x_test_input.iloc[:,:-1])), labels=['negative', 'positive']))
print('')
print('Precision: ', precision_score(y_test, model.predict(x_test_input.iloc[:,:-1])))
print('Recall: ', recall_score(y_test, model.predict(x_test_input.iloc[:,:-1])))
print('Accuracy: ', accuracy_score(y_test, model.predict(x_test_input.iloc[:,:-1])))
print('f1_score: ', f1_score(y_test, model.predict(x_test_input.iloc[:,:-1])))

---- XGboost model ----
                        pred_negative        pred_positive 
           true_negative       2077.0          0.0 
           true_positive       1716.0        207.0 
None

Precision:  1.0
Recall:  0.10764430577223089
Accuracy:  0.571
f1_score:  0.19436619718309858


# Training by bagging

In [19]:
from sklearn.linear_model import LogisticRegression
model1 = LogisticRegression()

from sklearn.tree import DecisionTreeClassifier
model2 = DecisionTreeClassifier()

from sklearn.neural_network import MLPClassifier
model3 = MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(1), random_state=1,max_iter=100000)

from sklearn import svm
model4 = svm.LinearSVC()

from sklearn.naive_bayes import MultinomialNB
model5 = MultinomialNB()

from sklearn.ensemble import RandomForestClassifier
model6 = RandomForestClassifier(n_estimators = 50,n_jobs = -1)

import xgboost as xgb
model7 = xgb.XGBClassifier()


In [20]:
model_list = [model1, model2, model3, model4, model5, model6, model7]
f1 = np.zeros([len(model_list)],dtype=np.float32)

In [21]:
for i,j in zip(model_list,range(len(model_list))):
    model = BaggingClassifierPU(i,
                         n_estimators = 50, 
                         n_jobs = -1, 
                         max_samples = sum(y)  
                        )
    model.fit(X,y)
    print(f1_score(y_orig, model.predict(X)))
    f1[j] = f1_score(y_orig, model.predict(X))

print(f1)

0.9542205682556559
0.8773440755660509
0.8045622073911743
0.8819129770158869
0.8039548586837113
0.9328929986789961
0.994773519163763
[0.9542206  0.8773441  0.8045622  0.881913   0.80395484 0.932893
 0.9947735 ]


In [22]:
f1_index = []
for i in range(len(f1)):
    if f1[i] >= 0.95:
        f1_index.append(i)
        
print(f1_index)
        
predict_sum = np.zeros([len(X)],dtype=np.float32)
for i in f1_index:
    model = BaggingClassifierPU(model_list[i],
                         n_estimators = 50, 
                         n_jobs = -1, 
                         max_samples = sum(y)  
                        )
    model.fit(X,y)
    predict_sum += model.predict(X)
print(predict_sum)

[0, 6]
[2. 1. 2. ... 2. 0. 0.]


In [23]:
threshold = len(f1_index) / 2
print(predict_sum)

predict = np.zeros([len(y)],dtype=np.int64)

for i in range(len(X)):
    if predict_sum[i] >= threshold :
        predict[i] = 1
    if predict_sum[i] < threshold :
        predict[i] = 0


print(predict)
print(y_orig)

[2. 1. 2. ... 2. 0. 0.]
[1 1 1 ... 1 0 0]
8118     1
10171    0
9399     1
12669    0
18809    0
        ..
13927    0
919      1
5699     1
10742    0
16921    0
Name: label, Length: 16000, dtype: int64


In [24]:
#train data
print('---- {} ----'.format('PU Bagging'))
print(print_cm(sklearn.metrics.confusion_matrix(y_orig,predict), labels=['negative', 'positive']))
print('')
print('Precision: ', precision_score(y_orig, predict))
print('Recall: ', recall_score(y_orig,predict))
print('Accuracy: ', accuracy_score(y_orig, predict))
print('f1_score: ', f1_score(y_orig, predict))

---- PU Bagging ----
                        pred_negative        pred_positive 
           true_negative       7148.0        775.0 
           true_positive          0.0       8077.0 
None

Precision:  0.9124491640307275
Recall:  1.0
Accuracy:  0.9515625
f1_score:  0.9542205682556559


In [25]:
best_model = BaggingClassifierPU(model_list[np.argmax(f1)],
                         n_estimators = 50, 
                         n_jobs = -1, 
                         max_samples = sum(y)  
                        )
print(best_model)

BaggingClassifierPU(base_estimator=XGBClassifier(base_score=None, booster=None,
                                                 colsample_bylevel=None,
                                                 colsample_bynode=None,
                                                 colsample_bytree=None,
                                                 gamma=None, gpu_id=None,
                                                 importance_type='gain',
                                                 interaction_constraints=None,
                                                 learning_rate=None,
                                                 max_delta_step=None,
                                                 max_depth=None,
                                                 min_child_weight=None,
                                                 missing=nan,
                                                 monotone_constraints=None,
                                                 n_estimators=1

In [26]:
print('Training bagging classifier...')

pu_start = time.perf_counter()

best_model.fit(X, y)
pu_end = time.perf_counter()
print('Done!')
print('Time:', pu_end - pu_start)


Training bagging classifier...
Done!
Time: 4.289725057024043


In [27]:
#train data
print('---- {} ----'.format('PU Bagging'))
print(print_cm(sklearn.metrics.confusion_matrix(y_orig, best_model.predict(X)), labels=['negative', 'positive']))
print('')
print('Precision: ', precision_score(y_orig, best_model.predict(X)))
print('Recall: ', recall_score(y_orig, best_model.predict(X)))
print('Accuracy: ', accuracy_score(y_orig, best_model.predict(X)))
print('f1_score: ', f1_score(y_orig, best_model.predict(X)))

---- PU Bagging ----
                        pred_negative        pred_positive 
           true_negative       7922.0          1.0 
           true_positive         83.0       7994.0 
None

Precision:  0.9998749218261413
Recall:  0.9897239073913582
Accuracy:  0.99475
f1_score:  0.994773519163763


In [28]:
#print wrong predictions
y_pre = best_model.predict(X)
y_orig_index = y_orig.index.tolist()

FN_index = []
FT_index = []

for i in range(len(y_orig)):
    if y_orig.iloc[i] == 1 and y_pre[i] == 0 :
        FN_index.append(y_orig_index[i])
    if y_orig.iloc[i] == 0 and y_pre[i] == 1 :
        FT_index.append(y_orig_index[i])
        
print("False Negtive:")
print(df_process.loc[FN_index])
print("False Positive:")
print(df_process.loc[FT_index])

False Negtive:
      p1  p2    p3  p4  label
917    1   0   306   0      1
894    0   0   297   1      1
9972   0   0  3323   1      1
875    1   0   292   0      1
116    1   0    39   0      1
...   ..  ..   ...  ..    ...
888    0   0   295   1      1
7846   0   1  2616   0      1
7843   0   1  2615   0      1
7854   0   0  2617   1      1
919    0   1   307   0      1

[83 rows x 5 columns]
False Positive:
       p1  p2  p3  p4  label
10002   0   1   0   0      0


In [29]:
#test data
print('---- {} ----'.format('PU Bagging'))
print(print_cm(sklearn.metrics.confusion_matrix(y_test, best_model.predict(x_test_input.iloc[:,:-1])), labels=['negative', 'positive']))
print('')
print('Precision: ', precision_score(y_test, best_model.predict(x_test_input.iloc[:,:-1])))
print('Recall: ', recall_score(y_test, best_model.predict(x_test_input.iloc[:,:-1])))
print('Accuracy: ', accuracy_score(y_test, best_model.predict(x_test_input.iloc[:,:-1])))
print('f1_score: ', f1_score(y_test, best_model.predict(x_test_input.iloc[:,:-1])))

---- PU Bagging ----
                        pred_negative        pred_positive 
           true_negative       2077.0          0.0 
           true_positive         26.0       1897.0 
None

Precision:  1.0
Recall:  0.9864794591783671
Accuracy:  0.9935
f1_score:  0.993193717277487


In [30]:
#print wrong predictions
y_test_pre = best_model.predict(x_test_input.iloc[:,:-1])
y_test_index = y_test.index.tolist()

FN_test_index = []
FT_test_index = []

for i in range(len(y_test)):
    if y_test.iloc[i] == 1 and y_test_pre[i] == 0 :
        FN_test_index.append(y_test_index[i])
    if y_test.iloc[i] == 0 and y_test_pre[i] == 1 :
        FT_test_index.append(y_test_index[i])

print("False Negtive:")
print(df_process.loc[FN_test_index])
print("False Positive:")
print(df_process.loc[FT_test_index])

False Negtive:
      p1  p2    p3  p4  label
7851   0   0  2616   1      1
923    1   0   308   0      1
906    0   0   301   1      1
928    0   1   310   0      1
9973   0   1  3325   0      1
878    1   0   293   0      1
929    1   0   310   0      1
903    0   0   300   1      1
7840   0   1  2614   0      1
900    0   0   299   1      1
908    1   0   303   0      1
7842   0   0  2613   1      1
885    0   0   294   1      1
921    0   0   306   1      1
9870   0   0  3289   1      1
915    0   0   304   1      1
115    0   1    39   0      1
926    1   0   309   0      1
912    0   0   303   1      1
9864   0   0  3287   1      1
9781   0   1  3261   0      1
922    0   1   308   0      1
118    0   1    40   0      1
7841   1   0  2614   0      1
9867   0   0  3288   1      1
9496   0   1  3166   0      1
False Positive:
Empty DataFrame
Columns: [p1, p2, p3, p4, label]
Index: []


In [31]:
orig_data = df_process.iloc[:,:-1].copy()
orig_label = best_model.predict(orig_data)

print(orig_data)
print(orig_label)

         p1    p2    p3    p4
0         1     0     0     0
1         0     1     1     0
2         1     0     1     0
3         0     0     0     1
4         0     1     2     0
...     ...   ...   ...   ...
19995  1948  3267  3248    22
19996   678  2207  1707  1330
19997   796  3271  1813  1477
19998  2163   614  2392  2879
19999  1420  2443  2561  1150

[20000 rows x 4 columns]
[1 1 1 ... 0 0 0]


In [32]:
import xgboost as xgb
model = BaggingClassifierPU(xgb.XGBClassifier(),
                         n_estimators = 50, 
                         n_jobs = -1, 
                         max_samples = sum(y)  
                        )
model.fit(orig_data, orig_label)

BaggingClassifierPU(base_estimator=XGBClassifier(base_score=None, booster=None,
                                                 colsample_bylevel=None,
                                                 colsample_bynode=None,
                                                 colsample_bytree=None,
                                                 gamma=None, gpu_id=None,
                                                 importance_type='gain',
                                                 interaction_constraints=None,
                                                 learning_rate=None,
                                                 max_delta_step=None,
                                                 max_depth=None,
                                                 min_child_weight=None,
                                                 missing=nan,
                                                 monotone_constraints=None,
                                                 n_estimators=1

In [33]:
#verification

import random

place_max = max(np.amax(df_process.iloc[:10000, [2]]))  #w_dependence
name1 = ['min', 'max', 'Precision', 'Recall', 'Accuracy', 'f1_score','time']
test = pd.DataFrame(columns=name1)


for i in range(1, 100):

    pu_start = time.perf_counter()
    
    pos_list = np.random.randint(low=0, high=10000, size=2000).tolist()
    neg_list = np.random.randint(low=10000, high=20000, size=2000).tolist()
    all_list = pos_list + neg_list
    
    flag1 = int((0.1 * (i - 1)) * place_max)
    flag2 = int((0.1 * i) * place_max)
    print(flag2)
    
    temp_data = df_process.iloc[all_list, : -1].copy()
    temp_label = df_process.iloc[all_list, -1].copy()
    
    temp_data.iloc[:, 2] = temp_data.iloc[:, 2] + random.randint((df_max[0] + flag1 + 1), ((df_max[0] + flag2 + 1)))
    
    print('======')
    print(temp_data.iloc[:, :])

    temp_max = max(np.amax(temp_data.iloc[:, [2]]))
    temp_min = min(np.amin(temp_data.iloc[:, [2]]))

    print('---- {} ----'.format(i))
    print('Precision: ', precision_score(temp_label, model.predict(temp_data)))
    print('Recall: ', recall_score(temp_label, model.predict(temp_data)))
    print('Accuracy: ', accuracy_score(temp_label, model.predict(temp_data)))
    print('f1_score: ', f1_score(temp_label, model.predict(temp_data)))
    
    
    orig_data = pd.concat([orig_data, temp_data], ignore_index=True)
    orig_label = pd.Series(orig_label.tolist() + model.predict(temp_data).tolist())
    model.fit(orig_data,orig_label)
    
    pu_end = time.perf_counter()

    test.loc[i] = [temp_min, temp_max,
                    precision_score(temp_label, model.predict(temp_data)),
                    recall_score(temp_label, model.predict(temp_data)),
                    accuracy_score(temp_label, model.predict(temp_data)),
                    f1_score(temp_label, model.predict(temp_data)),
                    (pu_end - pu_start)]
      
test.to_csv('../result/1_place_unknown_result.csv')

333
         p1    p2    p3    p4
835       0     1  3668     0
212       1     0  3460     0
9643      0     1  6604     0
2836      0     1  4335     0
4388      1     0  4852     0
...     ...   ...   ...   ...
18012   746  1552  4391  1450
16286   765  3247  3890  2149
12773     0     0  6072     0
18145  1511   257  4182  1026
19655  2633  2389  6266  1574

[4000 rows x 4 columns]
---- 1 ----
Precision:  1.0
Recall:  1.0
Accuracy:  1.0
f1_score:  1.0
666
       p1  p2    p3  p4
9008    1   0  6872   0
3028    0   1  4879   0
3109    0   1  4906   0
7933    0   1  6514   0
6054    0   0  5886   1
...    ..  ..   ...  ..
13545   0   0  6534   0
13317   1   0  5873   1
12829   0   1  6289   1
12639   1   1  4628   0
14838   1   1  4806   0

[4000 rows x 4 columns]
---- 2 ----
Precision:  1.0
Recall:  1.0
Accuracy:  1.0
f1_score:  1.0
999
         p1    p2    p3    p4
535       0     1  4454     0
803       1     0  4543     0
7418      1     0  6748     0
5601      0     0  6141     

Precision:  1.0
Recall:  1.0
Accuracy:  1.0
f1_score:  1.0
6332
         p1    p2     p3   p4
9822      0     0  12661    1
7355      1     0  11840    0
8082      0     0  12081    1
3456      0     0  10539    1
6231      0     0  11464    1
...     ...   ...    ...  ...
12562     1     1  10156    0
11262     0     1  10517    1
18603  2857  2062  10311  937
10957     0     1   9899    1
12079     1     0  12171    1

[4000 rows x 4 columns]
---- 19 ----
Precision:  1.0
Recall:  1.0
Accuracy:  1.0
f1_score:  1.0
6666
         p1    p2     p3    p4
6727      0     1  12208     0
9107      1     0  13001     0
7487      1     0  12461     0
1709      1     0  10535     0
1775      1     0  10557     0
...     ...   ...    ...   ...
17017  2430  1183  12656  2784
18949    57   813  11439  2391
12613     0     1  11654     1
14615     1     0  12089     1
18875  1447   221  12439  1727

[4000 rows x 4 columns]
---- 20 ----
Precision:  1.0
Recall:  1.0
Accuracy:  1.0
f1_score:  1.0
6999


Precision:  1.0
Recall:  1.0
Accuracy:  1.0
f1_score:  1.0
12332
         p1    p2     p3    p4
4582      0     1  17123     0
1202      1     0  15996     0
3049      0     1  16612     0
9129      0     0  18637     1
2584      0     1  16457     0
...     ...   ...    ...   ...
19532   125  2276  18534  1598
14453     1     1  18493     0
16416  2644  2444  16064  2798
17877  3024  1219  18313  1219
19512  1483   954  16600   181

[4000 rows x 4 columns]
---- 37 ----
Precision:  1.0
Recall:  1.0
Accuracy:  1.0
f1_score:  1.0
12665
         p1    p2     p3    p4
8416      0     1  18745     0
2010      0     0  16608     1
6441      0     0  18085     1
3036      0     0  16950     1
6260      1     0  18026     0
...     ...   ...    ...   ...
15852  2619  1697  17846  2404
17681   932  1751  16739  2266
10543     1     0  17679     1
11165     1     1  18848     1
16832    39  3135  17929  2675

[4000 rows x 4 columns]
---- 38 ----
Precision:  1.0
Recall:  1.0
Accuracy:  1.0
f1_sco

Precision:  1.0
Recall:  1.0
Accuracy:  1.0
f1_score:  1.0
17998
         p1    p2     p3    p4
6031      0     1  23286     0
9793      0     1  24540     0
3913      0     1  22580     0
1221      0     0  21681     1
8717      1     0  24181     0
...     ...   ...    ...   ...
16879  2723  2693  23191  2883
13546     0     1  23927     1
19037  2272  2415  21992   462
18292  2536  2552  21555  2630
11664     0     0  23341     0

[4000 rows x 4 columns]
---- 54 ----
Precision:  1.0
Recall:  1.0
Accuracy:  1.0
f1_score:  1.0
18331
         p1  p2     p3    p4
6823      0   1  23754     0
8042      1   0  24160     0
1941      0   0  22125     1
7195      0   1  23878     0
917       1   0  21785     0
...     ...  ..    ...   ...
15464  1401  36  23008  3000
14116     1   1  24218     0
13677     1   1  24370     0
10479     1   1  24600     1
12297     1   0  22624     1

[4000 rows x 4 columns]
---- 55 ----
Precision:  1.0
Recall:  1.0
Accuracy:  1.0
f1_score:  1.0
18664
         

Precision:  0.9995002498750625
Recall:  1.0
Accuracy:  0.99975
f1_score:  0.9997500624843789
23664
         p1    p2     p3   p4
9966      0     0  30180    1
3750      0     0  28108    1
6688      0     1  29089    0
1847      1     0  27475    0
9077      1     0  29885    0
...     ...   ...    ...  ...
10862     1     0  27703    1
14145     0     0  28971    0
10534     1     1  28029    0
19388  2143  2764  29171  494
17855  3066  1900  28086  137

[4000 rows x 4 columns]
---- 71 ----
Precision:  1.0
Recall:  1.0
Accuracy:  1.0
f1_score:  1.0
23997
         p1    p2     p3    p4
4589      1     0  28707     0
641       1     0  27391     0
9235      0     1  30256     0
226       0     1  27253     0
1747      0     1  27760     0
...     ...   ...    ...   ...
15973   474  2833  29858  1185
14348     1     0  30281     1
17303  2156   836  28127  1865
14931     1     0  30375     1
19398   383  1794  29091  3079

[4000 rows x 4 columns]
---- 72 ----
Precision:  1.0
Recall:  1.0

Recall:  1.0
Accuracy:  1.0
f1_score:  1.0
29330
         p1   p2     p3    p4
3341      1    0  33478     0
28        0    1  32374     0
7848      0    0  34979     1
9890      1    0  35661     0
28        0    1  32374     0
...     ...  ...    ...   ...
14531     0    1  32397     1
15501  1455  573  32822  2835
18581  2086  172  32970   477
14314     0    0  34094     0
13645     1    1  33136     1

[4000 rows x 4 columns]
---- 88 ----
Precision:  1.0
Recall:  1.0
Accuracy:  1.0
f1_score:  1.0
29663
         p1    p2     p3    p4
7571      1     0  35321     0
2806      0     1  33733     0
474       0     0  32954     1
7359      0     0  35249     1
6243      0     0  34877     1
...     ...   ...    ...   ...
11416     0     0  34571     0
17704  1003  2467  35582   775
11809     1     1  32845     1
18058  2072   561  33087   111
18201  2710  1659  35805  3062

[4000 rows x 4 columns]
---- 89 ----
Precision:  1.0
Recall:  1.0
Accuracy:  1.0
f1_score:  1.0
29997
         p1  